# Imports

In [1]:
import sys
import logging
import shutil
from uuid import uuid4
from tempfile import gettempdir
from os.path import join, isdir
import folium
from aequilibrae import Parameters, logger, Project
import pandas as pd

# Model place (Country) to build the model for

In [2]:
model_place = 'Andorra'

In [3]:
fldr = f'/home/jovyan/workspace/road_analytics/{model_place}'

if isdir(fldr):
    print(fldr)

/home/jovyan/workspace/road_analytics/Andorra


# Open Model

In [4]:
project = Project()
project.open(fldr)

# Get model data

In [5]:
links = pd.read_sql('select * from links', project.conn).drop(columns=['geometry'])

In [6]:
links.head()

,ogc_fid,link_id,a_node,b_node,direction,distance,modes,link_type,name,speed_ab,...,capacity_ab,capacity_ba,osm_id,bridge,tunnel,toll,surface,lanes_ab,lanes_ba,source_id
0,1,1,2,1,1,19.863131,ct,residential,None,NaN,...,None,None,1058565900,None,None,None,None,NaN,NaN,None
1,2,2,1,3,1,16.884412,ct,residential,None,NaN,...,None,None,1058565900,None,None,None,None,NaN,NaN,None
2,3,3,5,4,1,15.190793,ct,residential,None,NaN,...,None,None,1058565899,None,None,None,None,NaN,NaN,None
3,4,4,4,6,1,18.038848,ct,residential,None,NaN,...,None,None,1058565899,None,None,None,None,NaN,NaN,None
4,5,5,8,7,1,13.234459,ct,residential,None,NaN,...,None,None,1058514206,None,None,None,None,NaN,NaN,None


In [11]:
links.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9448 entries, 0 to 9447
Data columns (total 21 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ogc_fid      9448 non-null   int64  
 1   link_id      9448 non-null   int64  
 2   a_node       9448 non-null   int64  
 3   b_node       9448 non-null   int64  
 4   direction    9448 non-null   int64  
 5   distance     9448 non-null   float64
 6   modes        9448 non-null   object 
 7   link_type    9448 non-null   object 
 8   name         2936 non-null   object 
 9   speed_ab     1387 non-null   float64
 10  speed_ba     1084 non-null   float64
 11  capacity_ab  0 non-null      object 
 12  capacity_ba  0 non-null      object 
 13  osm_id       9448 non-null   int64  
 14  bridge       440 non-null    object 
 15  tunnel       76 non-null     object 
 16  toll         15 non-null     object 
 17  surface      3506 non-null   object 
 18  lanes_ab     1755 non-null   float64
 19  lanes_

---
Useful analytics:

* Surface type (pavement);
* Num. of nodes (per mode)
* Node density (per km²)
* Length of links (per mode)
* Density of links (per km²)
* Num.bridges
* Num. nodes corresponding to bridges
* Length of bridges
* Num. tunnels
* Num. nodes corresponding to tunnels
* Length of tunnels
* Num. tolls
* Num. nodes corresponding to toll roads
* Length of toll roads
* Avg. Speed
* Avg. num. of lanes
---

In [27]:
%%time
#Total km per surface type
df = pd.DataFrame(links[links["modes"].str.contains("c")].groupby('surface').sum(numeric_only=True)['distance'],
                  columns=['surface', 'total_km'])

df.head()

CPU times: user 14.6 ms, sys: 142 µs, total: 14.8 ms
Wall time: 15 ms


,surface,total_km


In [ ]:
links[links["modes"].str.contains("c")].groupby('link_type').sum(numeric_only=True)['distance']head

In [ ]:
# use geopandas area to calculate the country/region/area

project.close()